### Import Statements

In [46]:
import json
import cv2
import numpy as np
from ultralytics import YOLO

### Find Corners

In [48]:
def find_chessboard_corners(video_path, chessboard_size=(7, 7)):
    """
    Detect chessboard corners in the first frame of a video.

    Args:
        video_path (str): Path to the input video file.
        chessboard_size (tuple): Number of internal corners per chessboard row and column (rows, columns).

    Returns:
        bool: True if corners are found, False otherwise.
        numpy.ndarray: Coordinates of detected corners (if found).
        numpy.ndarray: The first frame with detected corners drawn.
    """
    # Open the video
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Error: Cannot open video file.")
        return False, None, None

    # Read the first frame from the video
    ret, frame = cap.read()
    cap.release()  # Release video resource
    if not ret:
        print("Error: Cannot read the first frame of the video.")
        return False, None, None

    # Convert frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Find chessboard corners
    found, corners = cv2.findChessboardCorners(gray_frame, chessboard_size, None)

    if found:
        # Define termination criteria
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        # Refine corner locations
        corners = cv2.cornerSubPix(gray_frame, corners, (11, 11), (-1, -1), criteria)
        # Draw corners on the frame
        frame_with_corners = cv2.drawChessboardCorners(frame, chessboard_size, corners, found)
        return True, corners, frame_with_corners
    else:
        print("Chessboard corners not found.")
        return False, None, frame



### Create JSON for Square positions

In [50]:
def map_positions_to_chessboard(corners, chessboard_size, output_json_path="chessboard_mapping.json"):
    """
    Map detected corner positions to squares on the chessboard and save the mapping to a JSON file.

    Args:
        corners (numpy.ndarray): Detected corner positions as a (N, 1, 2) array of points.
        chessboard_size (tuple): Number of internal corners per chessboard row and column (rows, columns).
        output_json_path (str): Path to save the chessboard mapping as a JSON file.

    Returns:
        dict: Mapping of chessboard squares to corner coordinates, e.g., {"A1": (x, y)}.
    """
    rows, cols = chessboard_size
    if len(corners) != rows * cols:
        raise ValueError("Number of detected corners does not match the expected chessboard size.")

    # Chessboard square mapping - bottom-left corner is A1
    chessboard_map = {}
    for row in range(rows):
        for col in range(cols):
            square = f"{chr(65 + col)}{row + 1}"  # Columns: A, B, C...; Rows: 1, 2, 3...
            index = row * cols + col
            position = tuple(map(float, corners[index, 0]))  # Convert numpy.float32 to float
            chessboard_map[square] = position

    # Save mapping - JSON 
    with open(output_json_path, "w") as json_file:
        json.dump(chessboard_map, json_file, indent=4)
    print(f"Chessboard mapping saved to {output_json_path}")

    return chessboard_map


### Object Detection with YOLOv8 & Mapping of Pieces to Squares

In [81]:
def object_detection_with_mapping(input_video_path, output_video_path, model_path, board_mapping_path, output_mapping_json):
    """
    Perform object detection on a video, apply grayscale preprocessing, map detected objects to chessboard squares, and save the output.

    Args:
        input_video_path (str): Path to the input video.
        output_video_path (str): Path to save the output video.
        model_path (str): Path to the YOLO model file.
        board_mapping_path (str): Path to the JSON file containing chessboard mapping.
        output_mapping_json (str): Path to save the detected piece-square mappings as a JSON file.
    """
    # Load YOLO model
    model = YOLO(model_path)

    # Load chessboard mapping
    with open(board_mapping_path, "r") as f:
        board_mapping = json.load(f)

    # Open input video
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define codec & create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # Initialize list to store mappings 
    all_frame_mappings = []

    # Perform object detection and mapping
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Perform detection with YOLOv8 on grayscale
        # YOLO expects a 3-channel image, so we need to convert back to BGR
        preprocessed_frame = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2BGR)

        results = model(preprocessed_frame)
        detections = results[0].boxes.data  # Access detected objects

        # Initialize a dictionary for detected pieces in the current frame
        frame_piece_positions = {}

        # Process each detection
        for detection in detections:
            # Extract bounding box & class
            x1, y1, x2, y2, confidence, class_id = detection.cpu().numpy()
            center_x, center_y = (x1 + x2) / 2, (y1 + y2) / 2  # Center of bounding box

            # Find nearest chessboard square
            nearest_square = None
            min_distance = float("inf")
            for square, position in board_mapping.items():
                mapped_x, mapped_y = position
                distance = ((center_x - mapped_x) ** 2 + (center_y - mapped_y) ** 2) ** 0.5
                if distance < min_distance:
                    min_distance = distance
                    nearest_square = square

            if nearest_square:
                piece_type = f"Class_{int(class_id)}"  # Placeholder for detected class label
                frame_piece_positions[nearest_square] = piece_type

        # Save frame piece positions to list
        all_frame_mappings.append(frame_piece_positions)

        # Print detected pieces & their positions
        for square, piece in frame_piece_positions.items():
            print(f"Detected {piece} at {square}")

        # Visualize detections on frame
        frame_with_detections = results[0].plot()

        # Write the frame with detections to the output video
        out.write(frame_with_detections)

    # Save frame mappings to JSON file
    with open(output_mapping_json, "w") as json_file:
        json.dump(all_frame_mappings, json_file, indent=4)
    print(f"Piece-square mappings saved to {output_mapping_json}")

    cap.release()
    out.release()
    print(f"Processed video saved at {output_video_path}")


In [83]:
# Set paths for read and save
video_path = r"C:\Users\robot\Documents\OD.mp4"
chessboard_size = (7, 7)  # Adjust to the dimensions of your chessboard
output_video_path = r"C:\Users\robot\Documents\Output\chess_game_output.mp4"
model_path = r"C:\Users\robot\Documents\Y8Run2\best.pt"  # Path to your trained YOLO model
board_mapping_path = r"C:\Users\robot\Documents\Output\chessboard_mapping.json"  # Path to pre-saved chessboard mapping JSON
output_mapping_json = r"C:\Users\robot\Documents\Output\detected_pieces.json"  # Path to save piece-square mappings

# Step 1: Detect corners in first frame of video
found, corners, annotated_frame = find_chessboard_corners(video_path, chessboard_size)

if found:
    # Step 2: Map positions to chessboard squares and save to JSON
    json_output_path = r"C:\Users\robot\Documents\Output\chessboard_mapping.json"
    chessboard_map = map_positions_to_chessboard(corners, chessboard_size, output_json_path=json_output_path)
    
    # Print mapping
    print("Chessboard mapping:")
    for square, position in chessboard_map.items():
        print(f"{square}: {position}")

    # Display the annotated frame
    cv2.imshow("Annotated Frame", annotated_frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Failed to detect chessboard corners.") # Debugging

# Step 3: Perform object detection & mapping
object_detection_with_mapping(video_path, output_video_path, model_path, board_mapping_path, output_mapping_json)

Chessboard mapping saved to C:\Users\robot\Documents\Output\chessboard_mapping.json
Chessboard mapping:
A1: (371.7565612792969, 312.5832824707031)
B1: (375.4472961425781, 331.4908447265625)
C1: (379.33587646484375, 351.5105895996094)
D1: (383.2347106933594, 374.54345703125)
E1: (387.7161560058594, 399.5608825683594)
F1: (392.94830322265625, 427.5452880859375)
G1: (399.3348083496094, 459.4858703613281)
A2: (331.2346496582031, 310.6827087402344)
B2: (332.52972412109375, 328.9006652832031)
C2: (333.8887634277344, 349.56488037109375)
D2: (335.9076843261719, 371.564453125)
E2: (337.6922912597656, 397.1270446777344)
F2: (340.52252197265625, 424.5157775878906)
G2: (342.766845703125, 455.6446228027344)
A3: (289.6357116699219, 308.562744140625)
B3: (289.5288391113281, 327.39190673828125)
C3: (289.442138671875, 347.6175842285156)
D3: (289.021240234375, 369.7320556640625)
E3: (288.2602233886719, 393.86895751953125)
F3: (287.4859924316406, 421.7523193359375)
G3: (287.10955810546875, 452.5964965820